In [1]:
%pylab inline

import os
import glob
import numpy as np
import astropy.table as astTable
import matplotlib.pyplot as plt
import xlens
import fitsio
import warnings
warnings.filterwarnings('ignore')

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
tplist = fitsio.read("/gpfs02/work/xiangchong.li/work/hsc_data/s23b/tracts_fdfc_v1_final_sims.fits")

In [3]:
colnames = [
    "flux", "fpfs_e1", "fpfs_e2", "wsel", "fpfs_de1_dg1", "fpfs_de2_dg2", "dwsel_dg1", "dwsel_dg2", "index",
    'g_modelfit_CModel_instFlux', 'g_modelfit_CModel_instFluxErr',
    'r_modelfit_CModel_instFlux', 'r_modelfit_CModel_instFluxErr',
    'i_modelfit_CModel_instFlux', 'i_modelfit_CModel_instFluxErr',
    'y_modelfit_CModel_instFlux', 'y_modelfit_CModel_instFluxErr',
    'z_modelfit_CModel_instFlux', 'z_modelfit_CModel_instFluxErr',
]
i = 0
data_all = []
for item in tplist:
    try:
        tract_id = item["tract"]
        patch_db = item["patch"]
        patch_x = patch_db // 100
        patch_y = patch_db % 100
        patch_id = patch_x + patch_y * 9
        match_fname = glob.glob(f"/work/xiangchong.li/work/hsc_s23b_sim/version1/anacal_match/20250823T193158Z/sim_0_rot0_Coadd_anacal_match/{tract_id}/{patch_id}/*")[0]
        force_fname = glob.glob(f"/work/xiangchong.li/work/hsc_s23b_sim/version1/anacal_force/20250820T011945Z/sim_0_rot0_Coadd_anacal_force/{tract_id}/{patch_id}/*")[0]
        match = astTable.Table.read(match_fname)[colnames]
        force = astTable.Table.read(force_fname)[match["index"]]
        out = astTable.hstack([match, force])
        data_all.append(out)
    except Exception:
        i+=1
        continue
print(i)


data_all = astTable.vstack(data_all, join_type="exact", metadata_conflicts="silent")
sel = np.logical_and(
    27 - 2.5 * np.log10(data_all["flux"]) < 24.5,
    data_all["fpfs_e1"]**2 + data_all["fpfs_e2"]**2 < 0.09,
    data_all["wsel"]>1e-6,
)
data_all = data_all[sel]

mag_g = 27 - 2.5 * np.log10(data_all["g_modelfit_CModel_instFlux"])
mag_r = 27 - 2.5 * np.log10(data_all["r_modelfit_CModel_instFlux"])
mag_i = 27 - 2.5 * np.log10(data_all["i_modelfit_CModel_instFlux"])
mag_z = 27 - 2.5 * np.log10(data_all["z_modelfit_CModel_instFlux"])
mag_y = 27 - 2.5 * np.log10(data_all["y_modelfit_CModel_instFlux"])

mag = 27 - 2.5 * np.log10(data_all["flux"])
mag_ga = 27 - 2.5 * np.log10(data_all["g_flux"])
mag_ra = 27 - 2.5 * np.log10(data_all["r_flux"])
mag_ia = 27 - 2.5 * np.log10(data_all["i_flux"])
mag_za = 27 - 2.5 * np.log10(data_all["z_flux"])
mag_ya = 27 - 2.5 * np.log10(data_all["y_flux"])

2763


ValueError: no values provided to stack.

In [ ]:
field_list = ["spring1", "spring2", "spring3", "autumn1", "autumn2", "hectomap"]
data_all = []
for field in field_list:
    out = []
    fname = f"/gpfs02/work/xiangchong.li/work/hsc_data/s23b/deepCoadd_anacal3/fields/{field}.fits"
    out.append(astTable.Table.read(fname))
    fname = f"/gpfs02/work/xiangchong.li/work/hsc_data/s23b/deepCoadd_anacal3/fields_color/{field}.fits"
    out.append(astTable.Table.read(fname))
    out = astTable.hstack(out)
    data_all.append(out)
data_all = astTable.vstack(data_all, join_type="exact", metadata_conflicts="silent")
sel = np.logical_and(
    27 - 2.5 * np.log10(data_all["flux"]) < 24.5,
    data_all["e1"]**2 + data_all["e2"]**2 < 0.09,
    data_all["wsel"]>1e-6,
)
data_all = data_all[sel]

mag_g_sim = data_all["g_cmodel_mag"]
mag_r_sim = data_all["r_cmodel_mag"]
mag_i_sim = data_all["i_cmodel_mag"]
mag_z_sim = data_all["z_cmodel_mag"]
mag_y_sim = data_all["y_cmodel_mag"]

mag_sim = 27 - 2.5 * np.log10(data_all["flux"])
mag_ga_sim = 27 - 2.5 * np.log10(data_all["g_flux"])
mag_ra_sim = 27 - 2.5 * np.log10(data_all["r_flux"])
mag_ia_sim = 27 - 2.5 * np.log10(data_all["i_flux"])
mag_za_sim = 27 - 2.5 * np.log10(data_all["z_flux"])
mag_ya_sim = 27 - 2.5 * np.log10(data_all["y_flux"])

In [ ]:
plt.close()
plt.hist(mag_g, bins=30, range=(20, 27), histtype="step", density =True, color="black", label="Data")
plt.hist(mag_g_sim, bins=30, range=(20, 27), histtype="step", density=True, color="black", linestyle="--", label="Sim")
plt.legend()
plt.xlabel(r"$i$ magnitude")
plt.ylabel("PDF")

In [ ]:
plt.close()
plt.hist(mag_r, bins=30, range=(20, 27), histtype="step", density =True, color="black", label="Data")
plt.hist(mag_r_sim, bins=30, range=(20, 27), histtype="step", density=True, color="black", linestyle="--", label="Sim")
plt.legend()
plt.xlabel(r"$i$ magnitude")
plt.ylabel("PDF")

In [ ]:
class corner_plot:
    def __init__(self, xrange, yrange, bins=30, xlabel="x", ylabel="y", xlim=None, ylim=None):
        fig = plt.figure(figsize=(8, 8))
        gs = fig.add_gridspec(4, 4, wspace=0.05, hspace=0.05)
        # 2D Contour plot (central plot)
        self.ax_2d = fig.add_subplot(gs[1:4, 0:3])
        self.ax_2d.set_xlabel(xlabel)
        self.ax_2d.set_ylabel(ylabel)
        
        self.ax_top = fig.add_subplot(gs[0, 0:3], sharex=self.ax_2d)
        self.ax_top.set_ylabel('PDF')
        self.ax_top.tick_params(axis="x", labelbottom=False)
        
        self.ax_right = fig.add_subplot(gs[1:4, 3], sharey=self.ax_2d)
        self.ax_right.set_xlabel('PDF')
        self.ax_right.tick_params(axis="y", labelleft=False)
        
        self.range_2d = [xrange, yrange]
        self.range_x = xrange
        self.range_y = yrange
        self.bins = bins

        if xlim is None:
            xlim = self.range_x
            
        if ylim is None:
            ylim = self.range_y
        self.ax_2d.set_xlim(xlim)
        self.ax_2d.set_ylim(ylim)
        self.ax_top.set_xlim(xlim)
        self.ax_right.set_ylim(ylim)
        return

    def make_plot(self, x_array, y_array, linestyles="-", levels=[0.1, 0.3, 0.6]):
        # Create figure and gridspec
        hist, xedges, yedges = np.histogram2d(
            x=x_array, y=y_array, bins=self.bins,
            range=self.range_2d,
            density=True,
        )
        # Calculate the midpoints of bins
        xcenters = 0.5 * (xedges[:-1] + xedges[1:])
        ycenters = 0.5 * (yedges[:-1] + yedges[1:])
        X, Y = np.meshgrid(xcenters, ycenters)
        
        # Contour plot
        contour = self.ax_2d.contour(
            X, Y, hist.T, levels=levels, 
            colors="black", linestyles=linestyles,
        )
        
        # 1D Histogram (top plot)
        self.ax_top.hist(x_array, bins=self.bins, histtype='step', color='black', 
            range=self.range_x, density=True, ls=linestyles,
        )
        
        # 1D Histogram (right plot)
        self.ax_right.hist(y_array, bins=self.bins, histtype='step', color='black', 
            range=self.range_y, density=True, 
            ls=linestyles, orientation='horizontal', 
        )
        
        # Final adjustments
        plt.tight_layout()

In [ ]:
plt.close()
xrange = [20.0, 26.0]
yrange = [-0.2, 1.3]
xlabel = r'$i$ magnitude'
ylabel = r'$r$ - $i$'

corp = corner_plot(
    xrange=xrange, yrange=yrange, bins=30, xlabel=xlabel, ylabel=ylabel,
    xlim=[20, 26.2], ylim=[-0.2, 1.3]
)

x_array = mag_i_sim
y_array = mag_r_sim - mag_i_sim

x_array_d = mag_i
y_array_d = mag_r - mag_i

corp.make_plot(x_array=x_array_d, y_array=y_array_d, linestyles="-",  levels=[0.1, 0.3, 0.65])
corp.make_plot(x_array=x_array, y_array=y_array, linestyles="--",  levels=[0.1, 0.3, 0.65])

In [ ]:
plt.close()

xrange = [20.0, 26.0]
yrange = [-0.5, 1.0]
xlabel = r'$i$ magnitude'
ylabel = r'$i$ - $z$'

corp = corner_plot(
    xrange=xrange, yrange=yrange, bins=30, xlabel=xlabel, ylabel=ylabel,
    xlim=[20, 26.2], ylim=[-0.5, 1.05]
)

x_array = mag_i_sim
y_array = mag_i_sim - mag_z_sim

x_array_d = mag_i
y_array_d = mag_i - mag_z

corp.make_plot(x_array=x_array_d, y_array=y_array_d, linestyles="-", levels=[0.15, 0.3, 0.65])
corp.make_plot(x_array=x_array, y_array=y_array, linestyles="--", levels=[0.15, 0.3, 0.65])

In [ ]:
plt.close()

xrange = [20., 26.2]
yrange = [0.2, 1.2]
xlabel = r'$i$ magnitude'
ylabel = r'$i$-band size [arcsec]'
corp = corner_plot(
    xrange=xrange, yrange=yrange, bins=30, xlabel=xlabel, ylabel=ylabel,
    xlim=[20., 26.2], ylim=[0.2, 1.06]
)

x_array = mag_sim[msk_sim]
y_array = size_sim[msk_sim]

x_array_d = mag_data[msk_data]
y_array_d = size_data[msk_data]

corp.make_plot(x_array=x_array_d, y_array=y_array_d, linestyles="-")
corp.make_plot(x_array=x_array, y_array=y_array, linestyles="--")